In [1]:
cd ..

E:\gdrive\Projects\Porto_Seguro\stacking


In [17]:
from utils import utils, gini, xgb_fn

import xgboost as xgb
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold

In [3]:
params_xgb_depth_5 = {}
params_xgb_depth_5['tree_method']= 'hist'
params_xgb_depth_5['silent']= True 
params_xgb_depth_5['eval_metric']= 'auc'
params_xgb_depth_5['objective']= 'binary:logistic'
params_xgb_depth_5['eta']= 0.01
params_xgb_depth_5['max_depth']= 5
params_xgb_depth_5['gamma']= 0.3
params_xgb_depth_5['min_child_weight']= 7
params_xgb_depth_5['subsample']= 0.7 
params_xgb_depth_5['colsample_bytree']= 0.6
params_xgb_depth_5['max_delta_step']= 5

In [6]:
combined = utils.load_data()
combined = utils.drop_stupid(combined)
combined = utils.cat_transform(combined, 'onehot')
train, test = utils.recover_train_test_na(combined, fillna=True)

In [7]:
y = pd.read_csv('data/train.csv').target

In [8]:
X = train.drop('target', axis=1)
y = train.target

In [9]:
sub_sample = {}
neg_train = train[train.target==0]
pos_index = train[train.target==1].index
for i in range(26):
    neg_index = neg_train.sample(n=21694, random_state=88).index
    neg_train = neg_train.drop(neg_index)
    sub_sample[str(i)] = np.concatenate((pos_index, neg_index))
    np.random.shuffle(sub_sample[str(i)])
    print('sub_sampling_set: {} length: {}'.format(i+1,len(sub_sample[str(i)])))

sub_sampling_set: 1 length: 43388
sub_sampling_set: 2 length: 43388
sub_sampling_set: 3 length: 43388
sub_sampling_set: 4 length: 43388
sub_sampling_set: 5 length: 43388
sub_sampling_set: 6 length: 43388
sub_sampling_set: 7 length: 43388
sub_sampling_set: 8 length: 43388
sub_sampling_set: 9 length: 43388
sub_sampling_set: 10 length: 43388
sub_sampling_set: 11 length: 43388
sub_sampling_set: 12 length: 43388
sub_sampling_set: 13 length: 43388
sub_sampling_set: 14 length: 43388
sub_sampling_set: 15 length: 43388
sub_sampling_set: 16 length: 43388
sub_sampling_set: 17 length: 43388
sub_sampling_set: 18 length: 43388
sub_sampling_set: 19 length: 43388
sub_sampling_set: 20 length: 43388
sub_sampling_set: 21 length: 43388
sub_sampling_set: 22 length: 43388
sub_sampling_set: 23 length: 43388
sub_sampling_set: 24 length: 43388
sub_sampling_set: 25 length: 43388
sub_sampling_set: 26 length: 43388


In [85]:
len(train_id)

34710

In [86]:
n_model = len(sub_sample)
n_folds = 5
oof_train = np.zeros((X.shape[0],))
oof_test = np.zeros((test.shape[0],))
rest_id = neg_train.index
xg_test = xgb.DMatrix(test)
xg_rest = xgb.DMatrix(X.iloc[rest_id,:])

for i in range(n_model):
    print('Starting Lap{}'.format(i+1))
    train_sample = sub_sample[str(i)]
    X_train = X.iloc[train_sample,:]
    y_train = y[train_sample]
    cross_val = StratifiedKFold(n_splits=n_folds, shuffle=False)
    avg_score=0
    
    for i, (train_index, val_index) in enumerate(cross_val.split(X_train, y_train)):
        
        train_id, val_id = train_sample[train_index], train_sample[val_index]
        X_train.reset_index(drop=True, inplace=True)
        y_train.reset_index(drop=True, inplace=True)
        X_tr, X_val = X_train.iloc[train_index,:], X_train.iloc[val_index,:]
        y_tr, y_val = y_train[train_index], y_train[val_index]

        y0_id = y_val[y_val==0]
        y1_id = y_val[y_val==1]
        val0_id = val_id[y0_id]
        val1_id = val_id[y1_id]
        
        xg_val0 = xgb.DMatrix(X_val.iloc[y0_id,:])
        xg_val1 = xgb.DMatrix(X_val.iloc[y1_id,:])

        # Set Var for XGB
        xg_train = xgb.DMatrix(X_tr, label=y_tr)
        xg_val = xgb.DMatrix(X_val, label=y_val)

        watchlist = [(xg_train, 'train'), (xg_val, 'test')]
        # train xgb
        xgb_skf = xgb.train(params_xgb_depth_5, xg_train, 10000, watchlist, early_stopping_rounds=100,
                            feval=gini.gini_xgb, maximize=True, verbose_eval=False)
        avg_score += xgb_skf.best_score / n_folds
        oof_train[val0_id] = xgb_skf.predict(xg_val0)
        oof_train[val1_id] += xgb_skf.predict(xg_val1) / 26
        oof_train[rest_id] += xgb_skf.predict(xg_rest) / 26
        oof_test += (xgb_skf.predict(xg_test)/n_folds) / 26

        print('[Fold {}/{} Gini score: {}]'.format(i+1, n_folds, xgb_skf.best_score))

Starting Lap1
[Fold 1/5 Gini score: 0.278109]
[Fold 2/5 Gini score: 0.27361]
[Fold 3/5 Gini score: 0.268049]
[Fold 4/5 Gini score: 0.282877]
[Fold 5/5 Gini score: 0.277852]
Starting Lap2
[Fold 1/5 Gini score: 0.278336]
[Fold 2/5 Gini score: 0.294458]
[Fold 3/5 Gini score: 0.290011]
[Fold 4/5 Gini score: 0.292839]
[Fold 5/5 Gini score: 0.274037]
Starting Lap3
[Fold 1/5 Gini score: 0.263516]
[Fold 2/5 Gini score: 0.283421]
[Fold 3/5 Gini score: 0.281438]
[Fold 4/5 Gini score: 0.27279]
[Fold 5/5 Gini score: 0.267812]
Starting Lap4
[Fold 1/5 Gini score: 0.297002]
[Fold 2/5 Gini score: 0.280562]
[Fold 3/5 Gini score: 0.278049]
[Fold 4/5 Gini score: 0.265694]
[Fold 5/5 Gini score: 0.270823]
Starting Lap5
[Fold 1/5 Gini score: 0.286512]
[Fold 2/5 Gini score: 0.261136]
[Fold 3/5 Gini score: 0.283706]
[Fold 4/5 Gini score: 0.272767]
[Fold 5/5 Gini score: 0.286966]
Starting Lap6
[Fold 1/5 Gini score: 0.26545]
[Fold 2/5 Gini score: 0.277613]
[Fold 3/5 Gini score: 0.279369]
[Fold 4/5 Gini score: 0

In [10]:
pd.DataFrame({'xgb_26': oof_test}).to_csv('oof/xgb_26_test', index=False)

In [56]:
gini.gini_sklearn(y, oof_train)

-0.010404305072809461

In [67]:
oof_train.item

<function ndarray.item>

In [20]:
n_model

26